# Tsunami Classification.

## Objectives:
1. Clean the data.
2. Statistical Analysis of data.
 - *Find frequency of tsunami by Year.*
 - *Find frequency of tsunami by Magnitude and Year.*
 - *Find correlation of tsunami and earthquake magnitude.*
 - *Geographically plotting occurance of tsunami by magnitude.*
3. Prediction using Decision Tree and Random Forrest.

## Import all libararies:
We import libraries as follows:

1. numpy for linear algebra
2. pandas for data processing
3. matplotlib, seaborn, shapely, geopandas for visualizarion
4. os for reading/writing data

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns  # visualization tool
from subprocess import check_output
import math

from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

## Reading the CSV file
The CSV file that we had generated earlier by extracting information from the tweets as mentioned in extraction python notebook

In [2]:
data = pd.read_csv('tsunami.csv')
#data.info()

## PreProcessing
Dropping rows that have the following columns empty

In [3]:

data.dropna(subset=['EarthquakeMagnitude'],inplace=True)
data.dropna(subset=['MaximumWaterHeight'],inplace=True)
data.dropna(subset=['Latitude'],inplace=True)
data.dropna(subset=['Longitude'],inplace=True)
data.head()

,Year,Mo,Dy,Hr,Mn,Sec,EarthquakeMagnitude,Deposits,Country,Location Name,Latitude,Longitude,MaximumWaterHeight
0,2008,2,25,8.0,36.0,33.0,6.5,0,INDONESIA,SUMATRA,-2.486,99.972,0.12
1,2008,4,9,12.0,46.0,12.7,7.3,0,NEW CALEDONIA,LOYALTY ISLANDS,-20.071,168.892,0.16
2,2008,4,28,18.0,33.0,34.2,6.4,0,VANUATU,VANUATU ISLANDS,-19.941,168.953,0.03
5,2008,7,19,2.0,39.0,28.7,6.9,0,JAPAN,OFF EAST COAST OF HONSHU ISLAND,37.552,142.214,0.20
8,2008,9,11,0.0,20.0,50.9,6.8,0,JAPAN,HOKKAIDO ISLAND,41.892,143.754,0.09


## Sorting the List by year
We sort the data by accesing order if the years. As you can see the output no year is repeated and is sorted in accending order

In [4]:
temp_years = list(set(data["Year"]))
unique_years = []
for year in temp_years:
    if year != '':
        unique_years.append(int(year))
    
unique_years.sort()
print(unique_years)

[2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]


## Calculating Quartiles
Calculate the lower quartiile, median and upper quartile for binning

In [7]:
print(data["MaximumWaterHeight"].quantile([0.25,0.5,0.75]))

0.25    0.09
0.50    0.19
0.75    0.50
Name: MaximumWaterHeight, dtype: float64


## Binning Data
We group the data based on Water Height:
1. Height less than 0.08
2. Height less than 0.20 and more than 0.08
3. Height less than 0.835 and more than 0.20
4. Height more than 0.835

We accordingly increment the count of the respective magnitudes(mentioned above) per year

In [8]:
#Group Earthquakes by no of occurance per year
year_mag_count = [0]*len(unique_years)
year_55_mag_count = [0]*len(unique_years)
year_65_mag_count = [0]*len(unique_years)
year_75_mag_count = [0]*len(unique_years)
year_100_mag_count = [0]*len(unique_years)

def get_year_index(data_year, unique_years):
    index = 0
    for year in unique_years:
        if year == data_year:
            return index
        index+=1
        
    return -1


for i in range(0, len(data)):
    mag = data["MaximumWaterHeight"][i]
    data_year = data["Year"][i]
    if data_year != '':
        year_index = get_year_index(int(data_year), unique_years)
        year_mag_count[year_index]+=1

        if float(mag) <= 0.08:
            year_55_mag_count[year_index]+=1
        elif float(mag) <= 0.20:
            year_65_mag_count[year_index]+=1
        elif float(mag) <= 0.835:
            year_75_mag_count[year_index]+=1
        else:
            year_100_mag_count[year_index]+=1
    
print(year_100_mag_count)


KeyError: 3

## Plotting Data
After the binning the data we visualize the data using a bargraph to identify patterns or trends that we can used to predict a natural disaster.

### We are plotting the count of the occurance of Tsunamis per year

In [ ]:
labels = unique_years

plt.rcParams.update({'font.size': 8})

x = np.arange(len(labels))  # the label locations
width = 0.8  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x, list(year_mag_count), width, align='edge')


# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Frequency')
ax.set_title('Number of Tsunamis(2008-2021)')
ax.set_xticks(x, labels)
for tick in ax.get_xticklabels():
    tick.set_rotation(90)
ax.legend()

ax.bar_label(rects1, padding=1)

fig.tight_layout()
fig.set_figwidth(10)
plt.show()


### We are plotting the count of the occurance of Tsunamis per year and magnitude

In [ ]:

labels = unique_years

plt.rcParams.update({'font.size': 8})

x = np.arange(len(labels))  # the label locations
width = 0.25  # the width of the bars

fig, ax = plt.subplots()
rects0 = ax.bar(x - 2 * width, list(year_55_mag_count), width, align='edge', label='Height <0.08m')
rects1 = ax.bar(x - width, list(year_65_mag_count), width, align='edge', label='Height (0.08m-0.2m)')
rects2 = ax.bar(x + width, list(year_75_mag_count), width, align='edge', label='Height (0.2m-0.83m)')
rects3 = ax.bar(x + 2* width, list(year_100_mag_count), width, align='edge', label='Height >0.83m')


# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Frequency')
ax.set_title('Comparision of magnitude scales Tsunamis(2008-2021)')
ax.set_xticks(x, labels)
for tick in ax.get_xticklabels():
    tick.set_rotation(90)
ax.legend()
ax.bar_label(rects0, padding=1)
ax.bar_label(rects1, padding=1)
ax.bar_label(rects2, padding=1)
ax.bar_label(rects3, padding=1)


fig.tight_layout()
fig.set_figwidth(10)
plt.show()


## Binning Data
We group the data based of Earthquake Magnitude based on Tsunami Height:
1. Height less than 0.08
2. Height less than 0.20 and more than 0.08
3. Height less than 0.835 and more than 0.20
4. Height more than 0.835

In [ ]:
#Group Earthquakes by no of occurance per year
year_dep_count = []
year_55_dep_count = []
year_65_dep_count = []
year_75_dep_count = []
year_85_dep_count = []
year_100_dep_count = []


for i in range(0, len(data)):
    mag = data["MaximumWaterHeight"][i]
    dep = data["EarthquakeMagnitude"][i]
    data_year = data["Year"][i]

    if data_year != '':

        if float(mag) <= 0.10:
            year_55_dep_count.append(dep)
        elif float(mag) <= 0.27:
            year_65_dep_count.append(dep)
        elif float(mag) <= 1.04:
            year_75_dep_count.append(dep)
        else:
            year_100_dep_count.append(dep)
    
print(year_65_dep_count)


## Plotting the Tsunami Height vs Earthquake Magnitude

This is use to find if there is any correlation between Tsunami Height and Earthquake Magnitude. We can see a linear increase therefore we can conclude that there is a linear correlation

In [ ]:
def Average(lst):
    x=0
    if len(lst)> 0:
        x = sum(lst) / len(lst)
        if not math.isnan(x):
            return int(x)
    return x

labels = ["<0.10", "<0.27", "<1.04",">1.04"]

plt.rcParams.update({'font.size': 8})

x = np.arange(len(labels))  # the label locations
width = 0.8  # the width of the bars

fig, ax = plt.subplots()
rects0 = ax.bar(x-width, [Average(year_55_dep_count),Average(year_65_dep_count),Average(year_75_dep_count),Average(year_100_dep_count)], width, align='edge', label='Depth')


# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_xlabel('Tsunami Height')
ax.set_ylabel('Earthquake Magnitude')
ax.set_title('Average Height of Tsunami and magnitude of earthquake(2000-2016)')
ax.set_xticks(x, labels)
for tick in ax.get_xticklabels():
    tick.set_rotation(90)
ax.legend()
ax.bar_label(rects0, padding=0)


fig.tight_layout()
fig.set_figwidth(3)
plt.show()


## Plotting the data on a world map

We visualize the regions that are affected by the earthquake and look for patterns that we can use to train the prediction model

In [ ]:
geometry = [Point(xy) for xy in zip([0,0,0], [0,0,0])]
gdf = GeoDataFrame([-2.486, -2.486, -2.486], geometry=geometry)   

#this is a simple map that goes with geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf.plot(ax=world.plot(figsize=(10, 6)), marker='*', color='green', markersize=5);

We are binning the earthquakes' latitude, longitude, depth and time based on the binning criteria mentioned earlier.

In [ ]:
lat55 = []
long55 = []

lat65 = []
long65 = []

lat75 = []
long75 = []

lat85 = []
long85 = []

lat100 = []
long100 = []

for i in range(0, len(data)):
    mag = data["MaximumWaterHeight"][i]
    lat = data["Latitude"][i]
    long = data["Longitude"][i]
    data_year = data["Year"][i]
    if data_year != '':

        if float(mag) <= 0.08:
            lat55.append(lat)
            long55.append(long)
        elif float(mag) <= 0.20:
            lat65.append(lat)
            long65.append(long)
        elif float(mag) <=  0.835:
            lat75.append(lat)
            long75.append(long)
        else:
            lat100.append(lat)
            long100.append(long)

Plotting on a world map the tsunamis with a height less than 0.08m

In [ ]:
geometry = [Point(xy) for xy in zip(long55, lat55)]
gdf = GeoDataFrame(lat55, geometry=geometry)   

#this is a simple map that goes with geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf.plot(ax=world.plot(figsize=(10, 6)), marker='*', color='blue', markersize=5);

Plotting on a world map the tsunamis with a height less than 0.2m greater than 0.08m

In [ ]:
geometry = [Point(xy) for xy in zip(long65, lat65)]
gdf = GeoDataFrame(lat65, geometry=geometry)   

#this is a simple map that goes with geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf.plot(ax=world.plot(figsize=(10, 6)), marker='*', color='yellow', markersize=5);

Plotting on a world map the tsunamis with a height less than 0.8m greater than 0.2m

In [ ]:
geometry = [Point(xy) for xy in zip(long75, lat75)]
gdf = GeoDataFrame(lat75, geometry=geometry)   

#this is a simple map that goes with geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf.plot(ax=world.plot(figsize=(10, 6)), marker='*', color='orange', markersize=5);

Plotting on a world map the tsunamis with a height greater than 0.8m

In [ ]:
geometry = [Point(xy) for xy in zip(long100, lat100)]
gdf = GeoDataFrame(lat100, geometry=geometry)   

#this is a simple map that goes with geopandas
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
gdf.plot(ax=world.plot(figsize=(10, 6)), marker='*', color='red', markersize=5);

Visualizing the average depth of an earthquake to the frequency of earthquakes per year

In [ ]:
# visualization
plt.figure(figsize=(10,5))
sns.barplot(y=year_55_mag_count[0:15], x=year_55_dep_count[0:15])
plt.xticks(rotation= 45)
plt.ylabel('Number of Earthquakes')
plt.xlabel('Average Depth')
plt.title('Depth of Earthquakes')